In [59]:
import csv

def count_diff(list_1, list_2):
    count = 0
    for x in range(0, len(list_1)):
        if list_1[x] != list_2[x]:
            count += 1
    return count





In [40]:
import numpy

#CHANGE THIS FOR PROB 1
for k in [3, 4, 5, 6, 7]:
    in_sample_data = []
    with open('in.dta.txt') as inp:
        for row in inp.readlines():
            row = row [:len(row)-1]
            row = row.split(' ')
            row1 = [x for x in row if x != '']
            row1[0] = float(row1[0])
            row1[1] = float(row1[1])
            if row1[2][0] == '-':
                row1[2] = -1
            else:
                row1[2] = 1
            in_sample_data.append(row1)

    out_sample_data = []
    with open('out.dta.txt') as inp:
        for row in inp.readlines():
            row = row [:len(row)-1]
            row = row.split(' ')
            row1 = [x for x in row if x != '']
            row1[0] = float(row1[0])
            row1[1] = float(row1[1])
            if row1[2][0] == '-':
                row1[2] = -1
            else:
                row1[2] = 1
            out_sample_data.append(row1)
            
    in_sample_class = []
    out_sample_class = []
    for set_number, data_set in enumerate([in_sample_data, out_sample_data]):
        for idx, point in enumerate(data_set):
            temp = [1, point[0], point[1], pow(point[0],2), pow(point[1],2), point[0]*point[1], abs(point[0]-point[1]), abs(point[0]+point[1])]
            data_set[idx] = temp[0:k+1]
            if set_number == 0:
                in_sample_class.append(point[2])
            else:
                out_sample_class.append(point[2])

    training = in_sample_data[0:25]
    validation = in_sample_data[25:] 
    training_class = in_sample_classification[0:25]
    validation_class = in_sample_classification[25:]

    X_points = numpy.array(validation)
    X_dagger = numpy.matmul(numpy.linalg.pinv(numpy.matmul(X_points.T, X_points)), X_points.T)
    w = numpy.matmul(X_dagger, numpy.array(validation_class))
    
    class_check = []
    for point in out_sample_data:
        if numpy.dot(point, w) > 0:
            class_check.append(1)
        else:
            class_check.append(-1)
                
    diff = count_diff(out_sample_classification, class_check)
    E_out = diff/len(out_sample_data)
    
    print(E_out)

0.396
0.388
0.284
0.192
0.196


In [16]:
from random import uniform
from statistics import mean
N = 100000
e_1 = [uniform(0, 1) for x in range(0, N)]
e_2 = [uniform(0,1) for x in range(0, N)]
e_min = []
for x in range(0, N):
    e_min.append(min(e_1[x], e_2[x]))
print(mean(e_1))
print(mean(e_2))
print(mean(e_min))


0.5002270303124802
0.49953240679252103
0.3336788429367848


In [80]:
from random import uniform
import numpy as np
import random
from sklearn import svm
        
def reclassify(w, class_list, points):
        for i in range(0, len(class_list)):
            if np.dot(points[i], w) > 0:
                class_list[i] = 1
            else:
                class_list[i] = -1
def sign(x):
    if x > 0:
        return 1
    else:
        return -1

trials = 1000
N = 10
sup_vector_avg = 0
percent = 0

for j in range(0, trials):
    d = 2
    true_class = [0]*N

#PLA PLA PLA PLA PLA
    while true_class == [0]*N or true_class == [1]*N or true_class == [-1]*N:
        x_1 = [uniform(-1, 1) for x in range(0, 2)]
        y_1 = [uniform(-1, 1) for x in range(0,2)]
        f_fit = np.polyfit(x_1, y_1, 1)

#TARGET FUNCTION
        def f(x):
            return x*f_fit[0] + f_fit[1]

        x_n = [uniform(-1, 1) for x in range(0, N)]
        y_n = [uniform(-1, 1) for x in range(0, N)]

        for i in range(0, N):
            if y_n[i] > f(y_n[i]):
                true_class[i] = 1
            else:
                true_class[i] = -1

#WEIGHT CALCULATION
    w = [0,0,0] # g(x) 

    classification = [0]*N
    points = [[1, x_n[i], y_n[i]] for i in range(0, N)]

    while true_class != classification:
        wrong = []
        for i in range(0, N):
            if classification[i] != true_class[i]:
                wrong.append(points[i])
        point = random.choice(wrong)
        idx = points.index(point)
        temp_sign = true_class[idx]
        w = [w[0] + temp_sign*point[0], w[1] + temp_sign*point[1], w[2] + temp_sign*point[2]]
        reclassify(w, classification, points)

#ITERATION TRACKING AND SVM   
    if j%100 == 0:
        print(j)
    clf = svm.SVC()
    clf.fit(points, true_class)
    
    sup_vector_avg += len(clf.support_vectors_)/trials
    
    x_large = [[1, uniform(-1, 1), uniform(-1, 1)] for i in range(0, 10000)]
    y_f = [f(x_large[i][1]) for i in range(len(x_large))]

    correct_class = [0]*10000

    for i in range(0,len(x_large)):
        if x_large[i][2] > y_f[i]:
            correct_class[i] = 1
        else:
            correct_class[i] = -1

    PLA_error = [sign(np.dot(x_large[i], w)) for i in range(0, 10000)]

    clf_guess = clf.predict(x_large)
    diff_svm = count_diff(clf_guess, correct_class)
    diff_PLA = count_diff(correct_class, PLA_error)
    E_out_PLA = diff_PLA/10000
    E_out_svm = diff_svm/10000
    if (E_out_svm < E_out_PLA):
        percent += 1/trials
print(percent)
print(sup_vector_avg)


0
100
200
300
400
500
600
700
800
900
0.4910000000000004
6.471999999999972


In [72]:
from sklearn import svm

trials = 1000
N = 10
sup_vector_avg = 0
percent = 0

for i in range(0, trials):
    if i%100 == 0:
        print(i)
    w, points, true_class = PLA(N)
    clf = svm.SVC()
    clf.fit(points, true_class)
    
    sup_vector_avg += len(clf.support_vectors_)/trials
    
    x_large = [[1, uniform(-1, 1), uniform(-1, 1)] for i in range(0, 1000)]
    y_f = [f(x_large[i][1]) for i in range(len(x_large))]

    correct_class = [0]*1000

    for i in range(0,len(x_large)):
        if x_large[i][2] > y_f[i]:
            correct_class[i] = 1
        else:
            correct_class[i] = -1

    PLA_error = [0]*1000
    for idx in range(0, len(PLA_error)):
        if np.dot(x_large[idx], w) > 0:
            PLA_error[idx] = 1
        else:
            PLA_error[idx] = -1

    clf_guess = clf.predict(x_large)
    diff_svm = count_diff(clf_guess, correct_class)
    diff_PLA = count_diff(correct_class, PLA_error)
    E_out_PLA = diff_PLA/1000
    E_out_svm = diff_svm/1000
    if (E_out_svm < E_out_PLA):
        percent += 1/trials
print(percent)

0
100
200
300
400
500
600
700
800
900
0.3990000000000003


In [71]:

x_large = [[1, uniform(-1, 1), uniform(-1, 1)] for i in range(0, 1000)]
y_f = [f(x_large[i][1]) for i in range(len(x_large))]

correct_class = [0]*1000

for i in range(0,len(x_large)):
    if x_large[i][2] > y_f[i]:
        correct_class[i] = 1
    else:
        correct_class[i] = -1

PLA_error = [0]*1000
for idx in range(0, len(PLA_error)):
    if np.dot(x_large[idx], w) > 0:
        PLA_error[idx] = 1
    else:
        PLA_error[idx] = -1

clf_guess = clf.predict(x_large)
diff_svm = count_diff(clf_guess, correct_class)
diff_PLA = count_diff(correct_class, PLA_error)
E_out_PLA = diff_PLA/1000
E_out_svm = diff_svm/1000

print(E_out_PLA)
print(E_out_svm)



0.122
0.129
